In [1]:
#Import Libraries
import pandas as pd
from pathlib import Path

In [2]:
#Import CSV with all data
data_path = Path("historical_data.csv")
historical_data = pd.read_csv(data_path)
historical_data.rename( columns = {"Unnamed: 0":"date"}, inplace = True)
historical_data.head()

,Date,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,10 YR
0,2008-01-02,32.13,28.34,79.75,28.41,34.94,38.37,41.33,26.10,41.98,3.91
1,2008-01-03,31.78,28.22,80.21,28.20,35.18,38.58,41.80,26.08,42.08,3.91
2,2008-01-04,30.78,28.04,78.01,27.39,34.83,37.60,40.74,25.07,42.30,3.88
3,2008-01-07,30.93,28.55,77.04,27.55,35.57,37.34,40.25,24.97,43.25,3.86
4,2008-01-08,30.32,28.38,76.08,26.54,35.81,36.39,39.60,24.21,43.33,3.86


In [3]:
# Define a function that returns the indexes where the 10 yr treasury momentum crossed a certain threshold
momentum_window=10
reaction_window=90
spike_percent= 0.187


def spike_selector(df):
    signal1 = 0
    daycount1 = 0
    indexes = []
    df["mmt10"] = (df["10 YR"] - df["10 YR"].shift(momentum_window))
    df["roll_10"] = df["mmt10"].rolling(20).mean()
    no_idx = df.reset_index()
    for index, row in zip(no_idx.index, no_idx["roll_10"]):
        if signal1 == 1:
            daycount1 += 1
            if daycount1 > reaction_window:
                signal1 = 0
                daycount1 = 0
        if row >= spike_percent and signal1 == 0:
            indexes.append(index)
            signal1 = 1
    df.drop(columns = ["mmt10", "roll_10"], inplace = True)
    return indexes



In [4]:
#Create empty dictionary to store new dataframes.  Then slice 90 days following each treaury yeild increase selected by the above function
yield_spikes_dict = {}
for index in spike_selector(historical_data):
    yield_spikes_dict[index] = pd.DataFrame
for index in spike_selector(historical_data):
    start = index
    end = start + 90
    yield_spikes_dict[index] = historical_data.iloc[start: end]

print(yield_spikes_dict.keys())

dict_keys([269, 360, 498, 736, 1357, 2221, 3292])


In [5]:
print(spike_selector(historical_data))


[269, 360, 498, 736, 1357, 2221, 3292]


In [6]:
#For loop to update keys
count = 1
final_dict={}
spike_list=[]
for key, value in yield_spikes_dict.items():
    new_df = yield_spikes_dict[key]
    new_df.reset_index(inplace=True)
    new_df.drop(columns=['Date','index'], axis=1, inplace=True)
    if new_df["10 YR"].count()>=85:
        spike_date=str(historical_data.iloc[key].Date)
        final_dict[spike_date]=new_df
        count +=1
    
print(final_dict.keys())

dict_keys(['2009-01-29', '2009-06-10', '2009-12-29', '2010-12-10', '2013-06-07', '2016-11-21'])


/Users/daviddivito/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
#Concat dfs for all spikes into one df, index by keys and export to pickle object
spikes_df= pd.concat(final_dict.values(), axis = 1, keys = final_dict.keys())
spikes_df.to_pickle('spikes_df.pickle')

In [8]:
# Calculate average percent change and groupby sector 
spikes_df.head()

spikes_pct_change= spikes_df.pct_change()
spikes_pct_change.to_pickle('spikes_pct_change.pickle')
    
grouped_df=spikes_pct_change.groupby(level = 1, axis = 1).mean()
grouped_df.head()
grouped_df.to_pickle('grouped_df.pickle')


spikes_pct_change.head()

2009-01-29                                                              \
         XLY       XLP       XLE       XLF       XLV       XLI       XLB   
0        NaN       NaN       NaN       NaN       NaN       NaN       NaN   
1  -0.026754 -0.031537 -0.013673 -0.022129 -0.012439 -0.021354 -0.039234   
2  -0.008817 -0.006784 -0.015782 -0.003233  0.009542 -0.022845 -0.013770   
3   0.028781  0.021403  0.017768 -0.023784  0.023819  0.018383  0.018296   
4  -0.022889 -0.026750  0.006813 -0.008859 -0.006278 -0.005330  0.016548   

                                 ... 2016-11-21                                \
        XLK       XLU     10 YR  ...        XLY       XLP       XLE       XLF   
0       NaN       NaN       NaN  ...        NaN       NaN       NaN       NaN   
1 -0.025132 -0.022965  0.000000  ...   0.012243  0.004916 -0.000618  0.000450   
2  0.014925  0.004839 -0.038328  ...   0.000968 -0.005675  0.004879  0.005843   
3  0.017380  0.007912  0.047101  ...   0.002537  0.007774 -0.005334  0.002234   
4 -0.000657  0.007850  0.020761  ...  -0.007714  0.002246 -0.014298 -0.013375   

                                                               
        XLV       XLI       XLB       XLK       XLU     10 YR  
0       NaN       NaN       NaN       NaN       NaN       NaN  
1 -0.013732  0.004529  0.006136  0.002717  0.003420 -0.008584  
2  0.003626  0.007348  0.004879 -0.003960 -0.009480  0.021645  
3  0.003757  0.005471  0.003035  0.004185  0.014195  0.000000  
4 -0.007774 -0.006925 -0.004236  0.000834  0.019722 -0.016949  

[5 rows x 60 columns]

In [9]:
#Calculate cumulative returns as percentage
returns_df = ((1+grouped_df).cumprod()-1) *100
returns_df.to_pickle('returns_df.pickle')


In [10]:
#Create df which groups the data by sector
sector_list = list(spikes_pct_change.columns.levels[1]) 
sector_dict={}
for sector in sector_list:
    sector_dict[str(sector)]=spikes_pct_change.xs(sector, level=1, axis=1)
sector_df= pd.concat(sector_dict.values(), axis = 1, keys = sector_dict.keys())
sector_df.to_pickle('sector_df.pickle')
sector_df.head()

XLY                                                         \
  2009-01-29 2009-06-10 2009-12-29 2010-12-10 2013-06-07 2016-11-21   
0        NaN        NaN        NaN        NaN        NaN        NaN   
1  -0.026754  -0.009136  -0.003972  -0.005057  -0.002816   0.012243   
2  -0.008817   0.001257  -0.009305  -0.000535  -0.009531   0.000968   
3   0.028781  -0.021766   0.006374  -0.003212  -0.011048   0.002537   
4  -0.022889  -0.022251   0.004000   0.007519   0.018018  -0.007714   

         XLP                                   ...        XLU             \
  2009-01-29 2009-06-10 2009-12-29 2010-12-10  ... 2009-12-29 2010-12-10   
0        NaN        NaN        NaN        NaN  ...        NaN        NaN   
1  -0.031537   0.000857  -0.001120   0.000000  ...   0.000318   0.005141   
2  -0.006784   0.005563  -0.011589   0.005493  ...  -0.014921   0.002238   
3   0.021403  -0.021277   0.010590   0.001366  ...   0.002256  -0.008612   
4  -0.026750  -0.011304  -0.001123   0.007842  ...  -0.011576   0.008687   

                             10 YR                                   \
  2013-06-07 2016-11-21 2009-01-29 2009-06-10 2009-12-29 2010-12-10   
0        NaN        NaN        NaN        NaN        NaN        NaN   
1  -0.001972   0.003420   0.000000  -0.025126  -0.005236  -0.009036   
2  -0.006061  -0.009480  -0.038328  -0.018041   0.013158   0.060790   
3  -0.010870   0.014195   0.047101  -0.013123   0.000000   0.011461   
4   0.017154   0.019722   0.020761  -0.023936  -0.020779  -0.016997   

                         
  2013-06-07 2016-11-21  
0        NaN        NaN  
1   0.023041  -0.008584  
2  -0.009009   0.021645  
3   0.022727   0.000000  
4  -0.026667  -0.016949  

[5 rows x 60 columns]